# Set-up

In [ ]:
# mount to google drive
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('./drive/MyDrive/Project')

In [ ]:
!pip install simplet5
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
!pip install accelerate -U

In [ ]:
import sacrebleu
import rouge_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch

import nltk
import evaluate

from datasets import load_dataset, DatasetDict, Dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
import tqdm
from tqdm.auto import tqdm
from simplet5 import SimpleT5

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# load data
df_train = pd.read_csv('./train_datasets/MedQuAD_train.csv')
df_test = pd.read_csv('./test_datasets/MedQuAD_test.csv')

# Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=123)

train_df = train_df.rename(columns={'Question': 'source_text', 'Answer_cut': 'target_text'})
val_df = val_df.rename(columns={'Question': 'source_text', 'Answer_cut': 'target_text'})

train_df['source_text'] = "[Question] " + train_df['source_text']
val_df['source_text'] = "[Question] " + val_df['source_text']

In [ ]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

In [ ]:
torch.cuda.empty_cache()


In [ ]:
model.train(train_df = train_df,
            eval_df = val_df,
            source_max_token_len=200,
            target_max_token_len=200,
            batch_size=32,
            max_epochs=2,
            use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Evaluation

In [ ]:
model = SimpleT5()
model.load_model("t5", "./outputs/MedQuAD-t5-checkpoints", use_gpu=torch.cuda.is_available())

q_test = df_test['Question'][48]
q_ans = df_test['Answer'][48]

print("Question: ", q_test)
print('-'*50)
print("Reference Answer: ",q_ans)
predicted_ans = model.predict(q_test)[0]
print("Predicted Answer: " ,predicted_ans)

Question:  What is (are) familial porencephaly ?
--------------------------------------------------
Reference Answer:  Familial porencephaly is part of a group of conditions called the COL4A1related disorders. The conditions in this group have a range of signs and symptoms that involve fragile blood vessels. In familial porencephaly, fluidfilled cysts develop in the brain (porencephaly) during fetal development or soon after birth. These cysts typically occur in only one side of the brain and vary in size. The cysts are thought to be the result of bleeding within the brain (hemorrhagic stroke). People with this condition also have leukoencephalopathy, which is a change in a type of brain tissue called white matter that can be seen with magnetic resonance imaging (MRI). During infancy, people with familial porencephaly typically have paralysis affecting one side of the body (infantile hemiplegia). Affected individuals may also have recurrent seizures (epilepsy), migraine headaches, spee

In [ ]:
model = SimpleT5()
model.load_model("t5", "./outputs/MedQuAD-t5-checkpoints", use_gpu=torch.cuda.is_available())

tokenizer = T5Tokenizer.from_pretrained("./outputs/MedQuAD-t5-checkpoints")
model = T5ForConditionalGeneration.from_pretrained("./outputs/MedQuAD-t5-checkpoints")
model.to(device)

df_test_MedQuAD = df_test

def generate_answers_batch(questions, batch_size=32):
    # initialize the answer list generated by batch processing
    batch_generated_answers = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Generating answers"):
        batch_questions = ["[Question] " + q for q in questions[i:i+batch_size]]
        batch_inputs = tokenizer(batch_questions, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        with torch.no_grad():
            batch_outputs = model.generate(**batch_inputs, max_length=500,
                                           min_length=50,
                                           length_penalty=2.0)
        batch_answers = [tokenizer.decode(output, skip_special_tokens=True) for output in batch_outputs]
        batch_generated_answers.extend(batch_answers)
    return batch_generated_answers



generated_answers = generate_answers_batch(df_test_MedQuAD['Question'].tolist())

df_test_MedQuAD['Generated_Answer'] = generated_answers

df_test_MedQuAD.to_csv("./outputs/MedQuADGenerated_Answer.csv", index = False)



Generating answers:   0%|          | 0/103 [00:00<?, ?it/s]

In [ ]:
references = [[ref_ans] for ref_ans in df_test_MedQuAD['Answer']]
predictions = [pre_ans for pre_ans in df_test_MedQuAD['Generated_Answer']]


sacrebleu = evaluate.load("sacrebleu")
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(f"SacreBLEU Results: {sacrebleu_results}")

rouge = evaluate.load("rouge")
rouge_results = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE Results: {rouge_results}")


SacreBLEU Results: {'score': 9.381689863930603, 'counts': [150482, 68891, 51156, 45296], 'totals': [600052, 596773, 593494, 590215], 'precisions': [25.07815989280929, 11.543920385138067, 8.619463718251575, 7.674491498860585], 'bp': 0.797510410279656, 'sys_len': 600052, 'ref_len': 735820}
ROUGE Results: {'rouge1': 0.24385678302336405, 'rouge2': 0.11190518722376439, 'rougeL': 0.2028155950496734, 'rougeLsum': 0.20265113194271184}
